<a href="https://colab.research.google.com/github/gurovic/MLCourse/blob/main/145_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 <a href="https://kaggle.com/kernels/welcome?src=https://github.com/gurovic/MLCourse/blob/main/145_ensemble.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>
       

# Ансамблевые методы в машинном обучении

## Подготовка данных для экспериментов

Перед изучением ансамблевых методов создадим два типа датасетов для экспериментов:


In [33]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# Нелинейный датасет (сложные границы решений)
X_moons, y_moons = make_moons(
    n_samples=500,
    noise=0.3,
    random_state=42
)


# Разделение на обучающую и тестовую выборки
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_moons, y_moons, test_size=0.2, random_state=42
)

## Основные концепции

**Ансамблевые методы** - подходы, которые комбинируют несколько базовых моделей для получения более точного и стабильного предсказания.

### Зачем нужны ансамбли?
1. Уменьшение дисперсии (variance reduction)
2. Увеличение точности предсказаний
3. Повышение устойчивости к шуму и выбросам
4. Компенсация слабостей отдельных моделей

### Основные типы ансамблей:
| Тип | Принцип | Примеры |
|------|---------|---------|
| **Бэггинг** | Параллельное обучение на разных подвыборках | Random Forest |
| **Бустинг** | Последовательное обучение с коррекцией ошибок | AdaBoost, XGBoost |
| **Стекинг** | Комбинирование предсказаний мета-моделью | Blending |

### Теорема Кондорсе (1785)
Основная математическая база для ансамблей: коллективное решение группы часто лучше индивидуальных решений, если каждый участник имеет вероятность правильного решения >0.5.

## 🟢 Базовый уровень

### 1.1 Принцип "Мудрости толпы"

In [34]:
import numpy as np

# Генерация предсказаний 100 слабых моделей (точность 51%)
true_labels = np.random.randint(0, 2, 1000)
weak_preds = [np.where(np.random.rand(1000) < 0.51, true_labels, 1-true_labels)
              for _ in range(100)]

# Агрегация предсказаний
ensemble_pred = np.mean(weak_preds, axis=0) > 0.5
ensemble_acc = np.mean(ensemble_pred == true_labels)
print(f"Точность ансамбля: {ensemble_acc:.2f}")  # ~75%

Точность ансамбля: 0.58


### 1.2 Виды ансамблей
- **Бэггинг (Bagging):** Bootstrap Aggregating
- **Бустинг (Boosting):** Последовательное улучшение
- **Стекинг (Stacking):** Мета-обучение

## 🟡 Продвинутый уровень

### 2.1 Реализация бэггинга


In [35]:
from sklearn.base import clone
from sklearn.tree import DecisionTreeClassifier

class BaggingClassifier:
    def __init__(self, base_estimator, n_estimators=10):
        self.estimators = [clone(base_estimator) for _ in range(n_estimators)]

    def fit(self, X, y):
        for estimator in self.estimators:
            indices = np.random.choice(len(X), size=len(X), replace=True)
            estimator.fit(X[indices], y[indices])

    def predict(self, X):
        preds = np.array([estimator.predict(X) for estimator in self.estimators])
        return np.round(np.mean(preds, axis=0))

# Использование
bagging = BaggingClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=50)
bagging.fit(X_train_m, y_train_m)

### 2.2 Сравнение типов ансамблей
| Метод | Параллелизм | Устойчивость к шуму | Скорость |
|-------|-------------|---------------------|----------|
| Бэггинг | ✅ Высокий | ✅ Высокая | ⚡ Быстрая |
| Бустинг | ❌ Последовательный | ❌ Чувствителен | 🐢 Медленная |
| Стекинг | ✅/❌ | ➖ Средняя | ⏱️ Зависит |

## 🔴 Экспертный уровень

### 3.1 Каскадное обучение


In [36]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

model = AdaBoostClassifier(
    estimator=SVC(probability=True, kernel='rbf'),
    n_estimators=100,
)
model.fit(X_train_m, y_train_m)

AdaBoostClassifier(estimator=SVC(probability=True), n_estimators=100)

### 3.2 Оптимизация стекинга


In [37]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

estimators = [
    ('dt', DecisionTreeClassifier(max_depth=5)),
    ('svm', SVC(probability=True)),
    ('knn', KNeighborsClassifier(n_neighbors=7))
]

stack = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    cv=5,
    stack_method='predict_proba'
)

## 📊 Сравнение методов

In [38]:
from sklearn.metrics import roc_auc_score

methods = {
    "Bagging": BaggingClassifier(
        base_estimator=DecisionTreeClassifier(),
        n_estimators=50
    ),
    "AdaBoost": AdaBoostClassifier(n_estimators=50),
    "Stacking": stack
}

for name, model in methods.items():
    model.fit(X_train_m, y_train_m)

    try:
        # Для моделей с вероятностями
        y_proba = model.predict_proba(X_test_m)[:, 1]
    except AttributeError:
        try:
            # Для моделей с decision_function
            y_proba = model.decision_function(X_test_m)
            # Линейное преобразование к [0, 1]
            y_proba = (y_proba - y_proba.min()) / (y_proba.max() - y_proba.min())
        except AttributeError:
            # Для моделей без вероятностей (используем жесткие предсказания)
            y_pred = model.predict(X_test_m)
            # Преобразуем в псевдо-вероятности (0.1 и 0.9 вместо 0 и 1)
            y_proba = np.where(y_pred == 0, 0.1, 0.9)

    score = roc_auc_score(y_test_m, y_proba)
    print(f"{name}: {score:.4f}")

Bagging: 0.9035
AdaBoost: 0.9810
Stacking: 0.9816


## ⚠️ Распространённые ошибки
1. Использование коррелированных моделей в ансамбле
2. Отсутствие разнообразия в базовых моделях
3. Неправильный выбор мета-модели для стекинга
4. Игнорирование переобучения при бустинге

## 📌 Заключение
- **Бэггинг:** Лучше для уменьшения дисперсии, параллельные вычисления
- **Бустинг:** Лучше для уменьшения смещения, последовательное улучшение
- **Стекинг:** Максимальная гибкость, требует тщательной настройки